In [2]:
!pip install openmim
!mim install mmdet
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -q -e .

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 518.6 kB/s eta 0:00:00a 0:00:01
Looking in links: https://download.openmmlab.com/mmcv/dist/cu110/torch1.11.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 4.1 MB/s eta 0:00:00a 0:00:010m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.1/575.1 kB 29.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mmcv-full: filename=mmcv_full-1.6.2-cp37-cp37m-linux_x86_64.whl size=28375153 sha256=901f3c17d6079800cc8d33e91d25745afbbe995979f758a94226c428f6446bca
  Stored in directory: /root/.cache/pip/wheels/51/0a/2c/990e1866b4c28e9a70ae3a0a304eeaf4711e85a87c647b8b79
  Created wheel for pycocotools: filename=pycocotools-2.0.5-cp37-cp37m-linux_x86_64.whl size=373761 sha256=b37578c8043473d188d4c9bd2006517741a1fd06b418f3ac9f2b43d639b0b4ad
  Stored in directory: /roo

In [7]:
%%writefile /kaggle/working/mmdetection/configs/_base_/datasets/didi_detection.py
dataset_type = 'CocoDataset'
data_root = '/kaggle/input/didi-detection/dataset/dataset_release/' # 更改数据集路径
test_root = '/kaggle/input/didi-detection/dataset/'

classes = ('car','van','bus','truck','person','bicycle','motorcycle','open-tricycle','closed-tricycle','forklift','large-block','small-block')

img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)

train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels']),
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]
data = dict(
    samples_per_gpu=2,
    workers_per_gpu=2,
    train=dict(
        type=dataset_type,
        classes=classes,
        ann_file=data_root + 'train.json',
        img_prefix=data_root + 'train/',
        pipeline=train_pipeline),
    val=dict(
        type=dataset_type,
        classes=classes,
        ann_file=data_root + 'val.json',
        img_prefix=data_root + 'val/',
        pipeline=test_pipeline),
    test=dict(
        type=dataset_type,
        classes=classes,
        ann_file=test_root + 'test.json',
        img_prefix=data_root + 'test/',
        pipeline=test_pipeline))
evaluation = dict(interval=1, metric='bbox')
# optimizer
optimizer = dict(type='SGD', lr=0.02, momentum=0.9, weight_decay=0.0001)
optimizer_config = dict(grad_clip=None)
# learning policy
lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=500,
    warmup_ratio=0.001,
    step=[8, 11])
runner = dict(type='EpochBasedRunner', max_epochs=1)

Writing /kaggle/working/mmdetection/configs/_base_/datasets/didi_detection.py


In [3]:
%%writefile /kaggle/working/mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_didi.py
_base_ = [
    '../_base_/models/faster_rcnn_r50_fpn.py',
    '../_base_/datasets/didi_detection.py',
    '../_base_/default_runtime.py'
]
model = dict(
    roi_head=dict(
        bbox_head=dict(
            num_classes=12,
        )
    )
)

Writing /kaggle/working/mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_didi.py


In [4]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv

In [5]:
!python /kaggle/working/mmdetection/tools/train.py /kaggle/working/mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_didi.py

/kaggle/working/mmdetection/mmdet/utils/setup_env.py:39: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting OMP_NUM_THREADS environment variable for each process '
/kaggle/working/mmdetection/mmdet/utils/setup_env.py:49: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting MKL_NUM_THREADS environment variable for each process '
2022-10-26 05:09:44,797 - mmdet - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) [GCC 9.4.0]
CUDA available: True
GPU 0: Tesla P100-PCIE-16GB
CUDA_HOME: /usr/local/cuda
NV

In [6]:
!ls /kaggle/working/mmdetection/configs/cascade_rcnn

README.md
cascade_mask_rcnn_r101_caffe_fpn_1x_coco.py
cascade_mask_rcnn_r101_caffe_fpn_mstrain_3x_coco.py
cascade_mask_rcnn_r101_fpn_1x_coco.py
cascade_mask_rcnn_r101_fpn_20e_coco.py
cascade_mask_rcnn_r101_fpn_mstrain_3x_coco.py
cascade_mask_rcnn_r50_caffe_fpn_1x_coco.py
cascade_mask_rcnn_r50_caffe_fpn_mstrain_3x_coco.py
cascade_mask_rcnn_r50_fpn_1x_coco.py
cascade_mask_rcnn_r50_fpn_20e_coco.py
cascade_mask_rcnn_r50_fpn_mstrain_3x_coco.py
cascade_mask_rcnn_x101_32x4d_fpn_1x_coco.py
cascade_mask_rcnn_x101_32x4d_fpn_20e_coco.py
cascade_mask_rcnn_x101_32x4d_fpn_mstrain_3x_coco.py
cascade_mask_rcnn_x101_32x8d_fpn_mstrain_3x_coco.py
cascade_mask_rcnn_x101_64x4d_fpn_1x_coco.py
cascade_mask_rcnn_x101_64x4d_fpn_20e_coco.py
cascade_mask_rcnn_x101_64x4d_fpn_mstrain_3x_coco.py
cascade_rcnn_r101_caffe_fpn_1x_coco.py
cascade_rcnn_r101_fpn_1x_coco.py
cascade_rcnn_r101_fpn_20e_coco.py
cascade_rcnn_r50_caffe_fpn_1x_coco.py
cascade_rcnn_r50_fpn_1x_coco.py
cascade_rcnn_r50_fpn_20e_coco.py
cascade_rcnn_x

In [7]:
!ls /kaggle/working/mmdetection/configs/_base_/models -a

.			      faster_rcnn_r50_fpn.py
..			      mask_rcnn_r50_caffe_c4.py
cascade_mask_rcnn_r50_fpn.py  mask_rcnn_r50_fpn.py
cascade_rcnn_r50_fpn.py       retinanet_r50_fpn.py
fast_rcnn_r50_fpn.py	      rpn_r50_caffe_c4.py
faster_rcnn_r50_caffe_c4.py   rpn_r50_fpn.py
faster_rcnn_r50_caffe_dc5.py  ssd300.py


In [11]:
%%writefile /kaggle/working/mmdetection/configs/cascade_rcnn/cascade_rcnw_fpn_1x_didi.py
_base_ = [
    '../_base_/models/cascade_rcnn_r50_fpn.py',
    '../_base_/datasets/didi_detection.py',
    '../_base_/default_runtime.py'
]
model = dict(
    roi_head=dict(
        bbox_head=[
            dict(
                type='Shared2FCBBoxHead',
                num_classes=12),
            dict(
                type='Shared2FCBBoxHead',
                num_classes=12),
            dict(
                type='Shared2FCBBoxHead',
                num_classes=12)
        ],
    )
)

Overwriting /kaggle/working/mmdetection/configs/cascade_rcnn/cascade_rcnn_r50_fpn_1x_didi.py


In [12]:
!python /kaggle/working/mmdetection/tools/train.py /kaggle/working/mmdetection/configs/cascade_rcnn/cascade_rcnn_r50_fpn_1x_didi.py

/kaggle/working/mmdetection/mmdet/utils/setup_env.py:39: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting OMP_NUM_THREADS environment variable for each process '
/kaggle/working/mmdetection/mmdet/utils/setup_env.py:49: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting MKL_NUM_THREADS environment variable for each process '
2022-10-26 05:40:59,398 - mmdet - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) [GCC 9.4.0]
CUDA available: True
GPU 0: Tesla P100-PCIE-16GB
CUDA_HOME: /usr/local/cuda
NV